<a href="https://colab.research.google.com/github/parkerburchett/Numerai/blob/Refactor-Max-DataPoints/Refactored_Max_Datapoints.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook explores the Hyper Parameter Space of the lightgbm library


You might want to find out how corrilataed different seeds of the optimal hyper params are. Then submit 4 versions of it, that are the most un  corrilated. but 3 nmr on each of them



In [ ]:
!pip install numerapi
import numerapi

In [ ]:
import numpy as np
import pandas as pd
import os, sys
import gc
import pathlib
import json
import datetime
from typing import List, NoReturn, Union, Tuple, Optional, Text, Generic, Callable, Dict
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, QuantileTransformer
from sklearn.model_selection import KFold, StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss, mean_squared_error, mean_absolute_error, f1_score
from scipy.stats import spearmanr # -P I think this is corr. 
import joblib

# model
import lightgbm as lgb
import xgboost as xgb
import operator

# visualize
import matplotlib.pyplot as plt
import matplotlib.style as style
import seaborn as sns
from matplotlib import pyplot
from matplotlib.ticker import ScalarFormatter
sns.set_context("talk")
style.use('seaborn-colorblind')



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Methods to Gather and Clean Incoming Data

In [ ]:
def create_global_variables()-> None:
  """
    Create all global variables. 
    ROUND_NUMBER,FEATURES,TARGET,
    TOURNAMENT_DATA,TRAINING_DATA,VALIDATION_DATA
  """
  try:
    if HAVE_GATHERED_DATA == FALSE:
      ping_training_data()
      ping_tournament_data()
      create_validation_data(df = TOURNAMENT_DATA)

      create_global_constants()
      drop_data_type_columns()
      HAVE_GATHERED_DATA = True
  except NameError:
      ping_training_data()
      ping_tournament_data()
      create_validation_data(df = TOURNAMENT_DATA)
      
      create_global_constants()
      drop_data_type_columns()
      HAVE_GATHERED_DATA = True

def drop_data_type_columns():
  TRAINING_DATA.drop(columns=["data_type"], inplace=True)
  VALIDATION_DATA.drop(columns=["data_type"], inplace=True) #
  TOURNAMENT_DATA.drop(columns=["data_type"], inplace=True)

def ping_training_data():
  global TRAINING_DATA
  TRAINING_DATA = read_data('train')
  
def ping_tournament_data():
  global TOURNAMENT_DATA
  TOURNAMENT_DATA = read_data('tournament')

def create_validation_data(df):
  global VALIDATION_DATA
  VALIDATION_DATA  = df[df["data_type"] == "validation"].reset_index(drop = True)

def cast_eras_as_int(x): 
    try:
        return int(x[3:]) # strip the first 3 characters from each era
    except:
        return -99

def read_data(data):
    if data == 'train':
        df = pd.read_csv('https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_training_data.csv.xz')
    elif data == 'tournament':
        df = pd.read_csv('https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_tournament_data.csv.xz')
        
    feature_cols = df.columns[df.columns.str.startswith('feature')]
    mapping = {0.0 : 0, 0.25 : 1, 0.5 : 2, 0.75 : 3, 1.0 : 4}

    for c in feature_cols:
        df[c] = df[c].map(mapping).astype(np.uint8)
        
    df["era"] = df["era"].apply(cast_eras_as_int)
    return df


def create_global_constants() -> None:
  global TARGET
  TARGET = get_target_constant(TOURNAMENT_DATA)
  global FEATURES
  FEATURES = get_features_constant(TOURNAMENT_DATA)
  napi = open_api_access()
  global ROUND_NUMBER
  ROUND_NUMBER = napi.get_current_round()


def get_target_constant(tournament_data: pd.DataFrame):
  return tournament_data.columns[tournament_data.columns.str.startswith('target')].values.tolist()[0]


def get_features_constant(tournament_data: pd.DataFrame):
  return [column_names for column_names in tournament_data.columns.values.tolist() if 'feature' in column_names]



def load_api_creds_into_dict():
  creds  = open('/content/drive/MyDrive/creds.json','r') 
  api_keys_dict = json.load(creds)
  creds.close()
  return api_keys_dict


def open_api_access():
    api_keys_dict = load_api_creds_into_dict()
    my_secret_key = api_keys_dict['secret_key']
    my_public_id = api_keys_dict['public_id']
    napi = numerapi.NumerAPI(secret_key=my_secret_key, public_id=my_public_id)
    return napi




In [ ]:
%%time
create_global_variables()
HAVE_GATHERED_DATA =True


## the first time you run this in a session it should take 20 minutes. 
## the second time it should be near instant.

CPU times: user 5min 12s, sys: 11.5 s, total: 5min 23s
Wall time: 5min 28s


### ModelStats Object

1. Stores the Trained Model
2. Stores the Hyper Parameters
3. Stores the Validation Scores

In [ ]:
class ModelStats():
  """
  An object that tracks Hyper Parameters, Time Costs and Scores. 
  """
  def __init__(self, model, scores:dict, total_time):
        self.model = model 
        self.hyperparams = model.get_params() 
        self.scores = scores 
        self.total_time = total_time
        self.all_stats_dict = None 


  def create_all_stats_dict(self):
    """
    Create a single dictionary tracking all the relevent statistics.
    """
    if self.all_stats_dict == None:
      
      all_stats_dict = {}
      all_stats_dict['total_time'] = self.total_time
      all_stats_dict['round_number'] = ROUND_NUMBER
      all_stats_dict.update(self.hyperparams) # dict.update(dict) merges two dictionaries
      all_stats_dict.update(self.scores)

      DECIMALS = 4 

      for key in all_stats_dict.keys():
          try:
            all_stats_dict[key] = [round(all_stats_dict[key],DECIMALS)]
          except:
            all_stats_dict[key] = [all_stats_dict[key]]

      self.all_stats_dict = all_stats_dict


  def headlines(self):
    """
    # Get a subset of scores that are the high level summary of the model
    """
    self.create_all_stats_dict()
    summary_dict = {}
    summary_dict['correlation'] = self.all_stats_dict['correlation']
    summary_dict['corr_sharpe'] = self.all_stats_dict['corr_sharpe']
    summary_dict['max_depth'] = self.all_stats_dict['max_depth']
    summary_dict['n_estimators'] = self.all_stats_dict['n_estimators']
    summary_dict['total_time'] = self.all_stats_dict['total_time']  
    summary_dict['num_leaves'] = self.all_stats_dict['num_leaves']
    summary_dict['learning_rate'] = self.all_stats_dict['learning_rate']
    return summary_dict
  

  def save_all_dict_to_disk(self):
    """
        Check to see if you already have scores saved. if yes. add these scores. 
        else create a new file to save scores
    """
    print('here')
    self.create_all_stats_dict()
    PATH_TO_SAVE_SCORES = '/content/drive/MyDrive/numerai_hyperparams_scores.csv'
    all_stats_df = pd.DataFrame.from_dict(my_model_stats.all_stats_dict, orient='index')
    print('1')

    if file_can_be_converted_to_df(PATH_TO_SAVE_SCORES):
      print('2')
      scores_already_saved_df = pd.read_csv(PATH_TO_SAVE_SCORES)
      print('5')
      scores_already_saved_df = pd.concat([scores_already_saved_df, all_stats_df], axis=1)
      scores_already_saved_df.to_csv(PATH_TO_SAVE_SCORES, index=False)
      print('4')

    else:
      with open(PATH_TO_SAVE_SCORES,'x'):
        pass
      all_stats_df.to_csv(PATH_TO_SAVE_SCORES, index=False)

def file_can_be_converted_to_df(path: str)-> bool:
  try:
    df = pd.read_csv(path)
    return True
  except:
    return False


def train_LGBMRegressor(params: dict, train_data): # there is not really a clear cell to put this method
  """
  Inputs: a dict of hyper paramaters for the model, 
  train_data: a pd.DataFrame of the Tiuraining Data

  Returns a trained model based on the parmas
  """
  model = lgb.LGBMRegressor(**params) 
  model.fit(train_data[FEATURES], train_data[TARGET])
  return model

#### Methods to Determine Validation Scores

1. I did not write these. I added the English comments

In [ ]:
# naming conventions
PREDICTION_NAME = 'prediction'
TARGET_NAME = TARGET # 'target is the string named 'target'
# EXAMPLE_PRED = 'example_prediction'

# ---------------------------
# Functions
# ---------------------------
def valid4score(valid : pd.DataFrame, pred : np.ndarray, load_example: bool=True, save : bool=False) -> pd.DataFrame:
    """
    Generate new valid pandas dataframe for computing scores
    
    :INPUT:
    - valid : pd.DataFrame extracted from tournament data (data_type='validation')
    
    """
    valid_df = valid.copy() # the validation dataframe you use this to test the CORR and other values

    # Your model creates an array of floats [0,1] rank method converst them in a list of ints. 

    # your lis tof ints is then compared to their list of ints. 
    valid_df['prediction'] = pd.Series(pred).rank(pct=True, method="first") # pred is the array of predictions your model creates for the set of validation vectors.  
    # I am unsure if this preds is a float only only between 0,1,2,3,4. 
    valid_df.rename(columns={TARGET: 'target'}, inplace=True)
    
    # I don't know what the load example boolean is. I think you can use this to save predictions.
    if load_example:
        valid_df[EXAMPLE_PRED] = pd.read_csv(EXP_DIR + 'valid_df.csv')['prediction'].values
    
    if save==True:
        valid_df.to_csv(OUTPUT_DIR + 'valid_df.csv', index=False)
        print('Validation dataframe saved!')
    
    return valid_df

def compute_corr(valid_df : pd.DataFrame):
    """
    Compute rank correlation

    THIS IS WHAT YOU ARE PRIMARILY PAID ON 
    
    :INPUT:
    - valid_df : pd.DataFrame where at least 2 columns ('prediction' & 'target') exist
    
    """
    # this uses Person Correilation. 
    # I You are paid on spearman corrilation. That is where the ratio of change is important not the raw amount of change
    # see: https://support.minitab.com/en-us/minitab-express/1/help-and-how-to/modeling-statistics/regression/supporting-topics/basics/a-comparison-of-the-pearson-and-spearman-correlation-methods/
    return np.corrcoef(valid_df["target"], valid_df['prediction'])[0, 1]

def compute_max_drawdown(validation_correlations : pd.Series):
    """
    Compute max drawdown
    
    :INPUT:
    - validation_correaltions : pd.Series
    """
    
    rolling_max = (validation_correlations + 1).cumprod().rolling(window=100, min_periods=1).max()
    daily_value = (validation_correlations + 1).cumprod()
    max_drawdown = -(rolling_max - daily_value).max()
    
    return max_drawdown

def compute_val_corr(valid_df : pd.DataFrame):
    """
    Compute rank correlation for valid periods
    
    :INPUT:
    - valid_df : pd.DataFrame where at least 2 columns ('prediction' & 'target') exist
    """
    
    # all validation
    correlation = compute_corr(valid_df)
    #print("rank corr = {:.4f}".format(correlation))
    return correlation
    
def compute_val_sharpe(valid_df : pd.DataFrame):
    """
    Compute sharpe ratio for valid periods
    
    :INPUT:
    - valid_df : pd.DataFrame where at least 2 columns ('prediction' & 'target') exist
    """
    # all validation
    d = valid_df.groupby('era')[['target', 'prediction']].corr().iloc[0::2,-1].reset_index()
    me = d['prediction'].mean()
    sd = d['prediction'].std()
    max_drawdown = compute_max_drawdown(d['prediction'])
    #print('sharpe ratio = {:.4f}, corr mean = {:.4f}, corr std = {:.4f}, max drawdown = {:.4f}'.format(me / sd, me, sd, max_drawdown))
    
    return me / sd, me, sd, max_drawdown
    
def feature_exposures(valid_df : pd.DataFrame):
    """
    Compute feature exposure
    
    :INPUT:
    - valid_df : pd.DataFrame where at least 2 columns ('prediction' & 'target') exist
    """
    feature_names = [f for f in valid_df.columns
                     if f.startswith("feature")]
    exposures = []
    for f in feature_names:
        fe = spearmanr(valid_df['prediction'], valid_df[f])[0]
        exposures.append(fe)
    return np.array(exposures)

def max_feature_exposure(fe : np.ndarray):
    return np.max(np.abs(fe))

def feature_exposure(fe : np.ndarray):
    return np.sqrt(np.mean(np.square(fe)))

def compute_val_feature_exposure(valid_df : pd.DataFrame):
    """
    Compute feature exposure for valid periods
    
    :INPUT:
    - valid_df : pd.DataFrame where at least 2 columns ('prediction' & 'target') exist
    """
    # all validation
    fe = feature_exposures(valid_df)
    fe1, fe2 = feature_exposure(fe), max_feature_exposure(fe)
    #print('feature exposure = {:.4f}, max feature exposure = {:.4f}'.format(fe1, fe2))
     
    return fe1, fe2

# to neutralize a column in a df by many other columns
#         I have no idea what this method does. -P. need to read about it and write up a link to it. 
def neutralize(df, columns, by, proportion=1.0):
    scores = df.loc[:, columns]
    exposures = df[by].values

    # constant column to make sure the series is completely neutral to exposures
    exposures = np.hstack(
        (exposures,
         np.asarray(np.mean(scores)) * np.ones(len(exposures)).reshape(-1, 1)))

    scores = scores - proportion * exposures.dot(
        np.linalg.pinv(exposures).dot(scores))
    return scores / scores.std()


# to neutralize any series by any other series
def neutralize_series(series, by, proportion=1.0):
    scores = series.values.reshape(-1, 1)
    exposures = by.values.reshape(-1, 1)

    # this line makes series neutral to a constant column so that it's centered and for sure gets corr 0 with exposures
    exposures = np.hstack(
        (exposures,
         np.array([np.mean(series)] * len(exposures)).reshape(-1, 1)))

    correction = proportion * (exposures.dot(
        np.linalg.lstsq(exposures, scores, rcond=None)[0]))
    corrected_scores = scores - correction
    neutralized = pd.Series(corrected_scores.ravel(), index=series.index)
    return neutralized


def unif(df):
    x = (df.rank(method="first") - 0.5) / len(df)
    return pd.Series(x, index=df.index)

def get_feature_neutral_mean(df):
    feature_cols = [c for c in df.columns if c.startswith("feature")]
    df.loc[:, "neutral_sub"] = neutralize(df, [PREDICTION_NAME],
                                          feature_cols)[PREDICTION_NAME]
    scores = df.groupby("era").apply(
        lambda x: np.corrcoef(x["neutral_sub"].rank(pct=True, method="first"), x[TARGET_NAME])).mean()
    return np.mean(scores)

def compute_val_mmc(valid_df : pd.DataFrame):    
    # MMC over validation
    mmc_scores = []
    corr_scores = []
    for _, x in valid_df.groupby("era"):
        series = neutralize_series(pd.Series(unif(x[PREDICTION_NAME])),
                                   pd.Series(unif(x[EXAMPLE_PRED])))
        mmc_scores.append(np.cov(series, x[TARGET_NAME])[0, 1] / (0.29 ** 2)) # I have no idea what htis line does (0.29 ** 2)
        corr_scores.append(np.corrcoef(unif(x[PREDICTION_NAME]).rank(pct=True, method="first"), x[TARGET_NAME]))

    val_mmc_mean = np.mean(mmc_scores)
    val_mmc_std = np.std(mmc_scores)
    val_mmc_sharpe = val_mmc_mean / val_mmc_std
    corr_plus_mmcs = [c + m for c, m in zip(corr_scores, mmc_scores)]
    corr_plus_mmc_sharpe = np.mean(corr_plus_mmcs) / np.std(corr_plus_mmcs)
    corr_plus_mmc_mean = np.mean(corr_plus_mmcs)

    #print("MMC Mean = {:.6f}, MMC Std = {:.6f}, CORR+MMC Sharpe = {:.4f}".format(val_mmc_mean, val_mmc_std, corr_plus_mmc_sharpe))

    # Check correlation with example predictions
    corr_with_example_preds = np.corrcoef(valid_df[EXAMPLE_PRED].rank(pct=True, method="first"),
                                          valid_df[PREDICTION_NAME].rank(pct=True, method="first"))[0, 1]
    #print("Corr with example preds: {:.4f}".format(corr_with_example_preds))
    
    return val_mmc_mean, val_mmc_std, corr_plus_mmc_sharpe, corr_with_example_preds


# this is the main method. The rest are just called interanlly. 
def score_summary(valid_df : pd.DataFrame):
    score_dict = {}
    
    try:
        score_dict['correlation'] = compute_val_corr(valid_df)
    except:
        print('ERR: computing correlation')
    try:
        score_dict['corr_sharpe'], score_dict['corr_mean'], score_dict['corr_std'], score_dict['max_drawdown'] = compute_val_sharpe(valid_df)
    except:
        print('ERR: computing sharpe')
    try:
        score_dict['feature_exposure'], score_dict['max_feature_exposure'] = compute_val_feature_exposure(valid_df)
    except:
        print('ERR: computing feature exposure')
    # try:
    #     score_dict['mmc_mean'], score_dict['mmc_std'], score_dict['corr_mmc_sharpe'], score_dict['corr_with_example_xgb'] = compute_val_mmc(valid_df)
    # except:
    #     print('ERR: computing MMC')
    
    return score_dict

### Main to train and track time, hyper parmas and scores.

In [ ]:
def train_validate_store(params, train_data, validation_data, features):
  """
  Train a model. Get Validation Scores. Create ModelStats Object. Save that model Stats object to your Google Drive
  """
  start_time = datetime.datetime.now()
  my_model = train_LGBMRegressor(params=params, train_data=train_data)
  my_predictions = my_model.predict(validation_data[features])
  valid_df = valid4score(validation_data, my_predictions, load_example=False, save=False)
  my_scores = score_summary(valid_df)
  total_time = (datetime.datetime.now() - start_time).total_seconds() 
  my_model_stats = ModelStats(model=my_model, scores=my_scores, total_time=total_time)
  my_model_stats.create_all_stats_dict()
  return my_model_stats

def create_DataFrame_from_ModelStats(modelStats_object: ModelStats)-> pd.DataFrame:
  df= pd.DataFrame.from_dict(modelStats_object.all_stats_dict)
  return df

def merge_df_vertically(old_stats_df : pd.DataFrame, new_stats_df: pd.DataFrame)-> pd.DataFrame:
  merged_df = pd.concat([old_stats_df, new_stats_df], axis=0)
  return merged_df
              

# Debugging

In [9]:
%%time
param_set=[]
for depth in range(3,5):
  for learning_rate in range(22,42,2):
    for feature_fraction in range(95,120,5):
      param_set.append({
            'n_estimators': 3000,
            'objective': 'regression',
            'boosting_type': 'gbdt',
            'max_depth': depth,
            'learning_rate': learning_rate / 1000,
            'feature_fraction': feature_fraction/ 1000, 
            'seed': 42 
              })

record_df = None

for param in param_set:
  my_model_stats = train_validate_store(param, 
                       train_data=TRAINING_DATA, 
                       validation_data = VALIDATION_DATA, 
                       features=FEATURES)
  if type(record_df) == None:
    record_df = create_DataFrame_from_ModelStats(my_model_stats) # also include a try pd.load(csv, index =false)
  else:
    new_df = create_DataFrame_from_ModelStats(my_model_stats)
    record_df = merge_df_vertically(record_df,new_df)
    # overwrite the record_df saved in drive
  record_df.to_csv(f'/content/drive/MyDrive/{ROUND_NUMBER}scores.csv', index=False)
  
  print(record_df.shape) # might also want to print the corr socres

2021-04-17 18:15:51,899 INFO numexpr.utils: NumExpr defaulting to 4 threads.


(1, 31)
(2, 31)
(3, 31)
(4, 31)
(5, 31)
(6, 31)
(7, 31)
(8, 31)
(9, 31)
(10, 31)
(11, 31)
(12, 31)
(13, 31)
(14, 31)
(15, 31)
(16, 31)
(17, 31)
(18, 31)
(19, 31)
(20, 31)
(21, 31)
(22, 31)
(23, 31)
(24, 31)
(25, 31)
(26, 31)
(27, 31)
(28, 31)
(29, 31)
(30, 31)
(31, 31)
(32, 31)
(33, 31)
(34, 31)
(35, 31)
(36, 31)
(37, 31)
(38, 31)
(39, 31)
(40, 31)
(41, 31)
(42, 31)
(43, 31)
(44, 31)
(45, 31)
(46, 31)
(47, 31)
(48, 31)
(49, 31)
(50, 31)
(51, 31)
(52, 31)
(53, 31)
(54, 31)
(55, 31)
(56, 31)
(57, 31)
(58, 31)
(59, 31)
(60, 31)
(61, 31)
(62, 31)
(63, 31)
(64, 31)
(65, 31)
(66, 31)
(67, 31)
(68, 31)
(69, 31)
(70, 31)
(71, 31)
(72, 31)
(73, 31)
(74, 31)
(75, 31)
(76, 31)
(77, 31)
(78, 31)
(79, 31)
(80, 31)
(81, 31)
(82, 31)
(83, 31)
(84, 31)
(85, 31)
(86, 31)
(87, 31)
(88, 31)
(89, 31)
(90, 31)
(91, 31)
(92, 31)
(93, 31)
(94, 31)
(95, 31)
(96, 31)
(97, 31)
(98, 31)
(99, 31)
(100, 31)
CPU times: user 12h 54min 42s, sys: 1min 5s, total: 12h 55min 48s
Wall time: 3h 33min 52s


In [10]:
record_df['correlation']

0    0.0243
0    0.0240
0    0.0244
0    0.0238
0    0.0229
      ...  
0    0.0232
0    0.0223
0    0.0220
0    0.0231
0    0.0231
Name: correlation, Length: 100, dtype: float64

In [24]:
def load_saved_params():
  df = pd.read_csv(f'/content/drive/MyDrive/{ROUND_NUMBER}scores.csv')
  return df


df = load_saved_params()
df.columns

Index(['total_time', 'round_number', 'boosting_type', 'class_weight',
       'colsample_bytree', 'importance_type', 'learning_rate', 'max_depth',
       'min_child_samples', 'min_child_weight', 'min_split_gain',
       'n_estimators', 'n_jobs', 'num_leaves', 'objective', 'random_state',
       'reg_alpha', 'reg_lambda', 'silent', 'subsample', 'subsample_for_bin',
       'subsample_freq', 'feature_fraction', 'seed', 'correlation',
       'corr_sharpe', 'corr_mean', 'corr_std', 'max_drawdown',
       'feature_exposure', 'max_feature_exposure'],
      dtype='object')

In [35]:
import statsmodels.api as sm

def compute_multiple_regression(
        df,
        indepenent_variables=['max_depth', 'learning_rate', 'feature_fraction'],
        dependent_variable='correlation'):
    x = df[indepenent_variables]
    y = df[dependent_variable]
    model = sm.OLS(y, x)
    results = model.fit()
    return results

res = compute_multiple_regression(df)
print(res.summary()
)

df[df.correlation == np.max(df.correlation)]


# it looks like the best for round 260 is seed=42, freature, fraction  =.095; n_estimators =3000, depth =4, learnign rate =

                                 OLS Regression Results                                
Dep. Variable:            correlation   R-squared (uncentered):                   0.995
Model:                            OLS   Adj. R-squared (uncentered):              0.995
Method:                 Least Squares   F-statistic:                              6766.
Date:                Sat, 17 Apr 2021   Prob (F-statistic):                   1.75e-112
Time:                        22:04:45   Log-Likelihood:                          498.71
No. Observations:                 100   AIC:                                     -991.4
Df Residuals:                      97   BIC:                                     -983.6
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                       coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------

,total_time,round_number,boosting_type,class_weight,colsample_bytree,importance_type,learning_rate,max_depth,min_child_samples,min_child_weight,min_split_gain,n_estimators,n_jobs,num_leaves,objective,random_state,reg_alpha,reg_lambda,silent,subsample,subsample_for_bin,subsample_freq,feature_fraction,seed,correlation,corr_sharpe,corr_mean,corr_std,max_drawdown,feature_exposure,max_feature_exposure
65,136.8332,260,gbdt,NaN,1.0,split,0.028,4,20,0.001,0.0,3000,-1,31,regression,NaN,0.0,0.0,1,1.0,200000,0,0.095,42,0.0259,0.9844,0.0264,0.0268,-0.0595,0.0716,0.2648


In [33]:
df[df.correlation > 0.024300]

,total_time,round_number,boosting_type,class_weight,colsample_bytree,importance_type,learning_rate,max_depth,min_child_samples,min_child_weight,min_split_gain,n_estimators,n_jobs,num_leaves,objective,random_state,reg_alpha,reg_lambda,silent,subsample,subsample_for_bin,subsample_freq,feature_fraction,seed,correlation,corr_sharpe,corr_mean,corr_std,max_drawdown,feature_exposure,max_feature_exposure
2,117.0181,260,gbdt,NaN,1.0,split,0.022,3,20,0.001,0.0,3000,-1,31,regression,NaN,0.0,0.0,1,1.0,200000,0,0.105,42,0.0244,0.9195,0.0248,0.0270,-0.0733,0.0838,0.2535
6,116.1439,260,gbdt,NaN,1.0,split,0.024,3,20,0.001,0.0,3000,-1,31,regression,NaN,0.0,0.0,1,1.0,200000,0,0.100,42,0.0248,0.9059,0.0253,0.0279,-0.0807,0.0817,0.2522
8,122.2655,260,gbdt,NaN,1.0,split,0.024,3,20,0.001,0.0,3000,-1,31,regression,NaN,0.0,0.0,1,1.0,200000,0,0.110,42,0.0245,0.9186,0.0250,0.0272,-0.0643,0.0815,0.2533
13,118.0555,260,gbdt,NaN,1.0,split,0.026,3,20,0.001,0.0,3000,-1,31,regression,NaN,0.0,0.0,1,1.0,200000,0,0.110,42,0.0244,0.8999,0.0248,0.0276,-0.0740,0.0803,0.2539
16,113.8561,260,gbdt,NaN,1.0,split,0.028,3,20,0.001,0.0,3000,-1,31,regression,NaN,0.0,0.0,1,1.0,200000,0,0.100,42,0.0245,0.9491,0.0249,0.0262,-0.0624,0.0801,0.2509
34,122.7018,260,gbdt,NaN,1.0,split,0.034,3,20,0.001,0.0,3000,-1,31,regression,NaN,0.0,0.0,1,1.0,200000,0,0.115,42,0.0246,1.0015,0.0250,0.0250,-0.0453,0.0762,0.2441
37,109.8449,260,gbdt,NaN,1.0,split,0.036,3,20,0.001,0.0,3000,-1,31,regression,NaN,0.0,0.0,1,1.0,200000,0,0.105,42,0.0245,0.9402,0.0250,0.0266,-0.0464,0.0748,0.2498
38,117.1597,260,gbdt,NaN,1.0,split,0.036,3,20,0.001,0.0,3000,-1,31,regression,NaN,0.0,0.0,1,1.0,200000,0,0.110,42,0.0245,0.9372,0.0250,0.0266,-0.0616,0.0753,0.2475
39,117.5454,260,gbdt,NaN,1.0,split,0.036,3,20,0.001,0.0,3000,-1,31,regression,NaN,0.0,0.0,1,1.0,200000,0,0.115,42,0.0245,0.9365,0.0250,0.0267,-0.0600,0.0751,0.2488
44,116.6783,260,gbdt,NaN,1.0,split,0.038,3,20,0.001,0.0,3000,-1,31,regression,NaN,0.0,0.0,1,1.0,200000,0,0.115,42,0.0245,0.9518,0.0249,0.0262,-0.0459,0.0738,0.2512


In [13]:
# # create histogram of corrlation scores by random seed. 
# final_scores = record_df['correlation']
# f = final_scores.to_numpy()
# std_dev_of_corr = round(np.std(f),4)
# avg_corr = round(np.average(f),4)

# sharpe_of_corr = round(avg_corr / std_dev_of_corr,4).

# # interestingly these scores are very tightly coupled.  random seed does not tend to have much influcence
# print(f'The average corr was {avg_corr} with a Standard Deviation of {std_dev_of_corr} with a sharpe of {sharpe_of_corr}')

# Submit to Numerai

I don't think I should submit anything to Numerai here. 

1. Create a prediction list.
2. Link those predictions with the tournment data
3. Write the id, prediction to a csv file.
4. Use numerai wrapper to submit that .csv file as your current model. 
5. This submits for MRQUANTSALOT and TUTMODEL



### Methods to handle submission

In [14]:
def load_api_creds_into_dict(): # works
  """
    Read creds.json and return a dictionary of your API keys.
  """
  creds  = open('creds.json','r') # refactor this to point at your google drive. 
  api_keys_dict = json.load(creds)
  creds.close()
  return api_keys_dict


def open_api_access(): # works
    """
    Read in my private key from creds.json and return the numerai api wrapper
    """
    api_keys_dict = load_api_creds_into_dict()
    my_secret_key = api_keys_dict['secret_key']
    my_public_id = api_keys_dict['public_id']
    napi = numerapi.NumerAPI(secret_key=my_secret_key, public_id=my_public_id)
    return napi


def create_id_prediction_df(tournament_data: pd.DataFrame, model_predictions : np.ndarray): # works
    """
    Create a dataframe that looks like 
    id,prediction
    asdfads,.5429
    asdfaddsss,.5051
    ...
    """
    predictions_df = tournament_data["id"].to_frame() # get all the Ids and cast them to a Dataframe
    predictions_df[PREDICTION_NAME] = model_predictions #add your predictions to the id frame
    return predictions_df # data frame of id, prediction


def write_predictions_to_file(prediction_df: pd.DataFrame): # works
    my_file_name = 'myPredictions.csv'
    try:
      out_location = open(my_file_name, 'x')
    except:
      out_location = open(my_file_name, 'w')

    prediction_df.to_csv(out_location, index=False)
    out_location.close()
    return my_file_name 


def run_model_and_create_prediction_file(model_object, tournament_data: pd.DataFrame, features: list):
  """
    This stitches everything together.

    Pass it a trained model and the tournament data set, the list of feature columns
    1. Does preditions
    2. write the predictions to a file.
    3. returns the name fo the file where my predictions are saved data is saved
  """
  model_predictions = model_object.predict(tournament_data[features])
  prediction_df = create_id_prediction_df(tournament_data,model_predictions)
  file_with_predictions = write_predictions_to_file(prediction_df)
  return file_with_predictions


def submit_predictions_to_numerai(filename_of_predictions, sumbit_model_id):
    napi = open_api_access() # open a connection to the numerai API with your creds.json file
    submission_id = napi.upload_predictions(filename_of_predictions, model_id=sumbit_model_id)
    print(f'You successfully submitted for {sumbit_model_id}')
    print(type(submission_id))
    return submission_id

print('your helper methods work correctly')
    

your helper methods work correctly
